In [1]:
from datasets import load_dataset, load_from_disk, concatenate_datasets
import os

/disks/local/c.4t_4/oscar/zhtw-dataset-cleaning/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
datasets = []
for dataset_name in os.listdir("instruction_tuning"):
    if not dataset_name.endswith(".jsonl"):
        continue
    print(dataset_name)
    datasets.append(load_dataset("json", data_files=f"instruction_tuning/{dataset_name}", split='train'))

custom_dataset = concatenate_datasets(datasets)
custom_dataset = custom_dataset.add_column('meta',  ["None"] * len(custom_dataset))
print(custom_dataset)
print(custom_dataset[0])

coct_en2tw.jsonl


Generating train split: 10389 examples [00:00, 106104.63 examples/s]


alpaca_zhtw_formatted.jsonl


Generating train split: 104004 examples [00:00, 210543.07 examples/s]


idiom.jsonl


Generating train split: 111 examples [00:00, 26398.72 examples/s]


Dataset({
    features: ['id', 'conversations', 'meta'],
    num_rows: 114504
})
{'id': 1063099886867503327, 'conversations': [{'role': 'user', 'content': 'Kindly render the English passage below into Traditional Chinese: To meet the challenge posed by the Internet, the World Association of Newspapers met in Madrid last month. Representatives from around the world agreed that if the traditional newspaper industry is to turn the crisis triggered by the Internet into an opportunity for market leadership, it will have to embrace digital technology, expand web-based services, and develop new online profit models.'}, {'role': 'assistant', 'content': '針對網路對傳統報業的衝擊，上月初「世界報業協會」於西班牙馬德里舉行的會議裡，各國與會代表一致同意，惟有更積極擁抱數位科技，努力擴展網路服務，並開發新的線上獲利模式，傳統報業才有可能把網路所帶來的危機，轉化為自己的市場優勢。'}], 'meta': 'None'}


In [3]:
from collections import OrderedDict
from datasets import Dataset
import itertools
def sort_conv_dict(example):
    sorted_conversations = []
    for conv in example['conversations']:
        dict_ = OrderedDict()
        dict_['role'] = conv['role']
        dict_['content'] = conv['content']
        sorted_conversations.append(dict_)
    example['conversations'] = sorted_conversations
    return example


aya = load_from_disk('instruction_tuning/aya_dataset')

# initialize a dataset with content of both datasets
def gen():
    for i in itertools.chain(custom_dataset, aya):
        yield i
en2tw_dataset = Dataset.from_generator(gen)
en2tw_dataset.push_to_hub("en2tw-alignment-sft")

Generating train split: 119413 examples [00:09, 12954.63 examples/s]
Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.98s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/liswei/en2tw-alignment-sft/commit/4a190e6655d42a4432b6a39a28bc088c2c9ea803', commit_message='Upload dataset', commit_description='', oid='4a190e6655d42a4432b6a39a28bc088c2c9ea803', pr_url=None, pr_revision=None, pr_num=None)